In [2]:
import pandas as pd
from pathlib import Path

In [18]:
data_dir = Path().cwd().parent / 'data' / 'FBRef'

In [19]:
def get_fbref_data(data_dir):
    files = data_dir.glob('*.csv')
    data = {}
    for file in files:
        data[file.stem] = pd.read_csv(file)
    return data

In [20]:
big5_data = get_fbref_data(data_dir / 'big5')

In [26]:
big5_data['2020_M_misc_team'].memory_usage(deep=True).sum() / 1024**2

0.08900642395019531

In [34]:
team_stats = set([k.split("_")[2] + "_team" for k in big5_data.keys() if 'team' in k])
player_stats = set([k.split("_")[2] + "_player" for k in big5_data.keys() if 'player' in k])

In [56]:
def join_csvs(regex_pattern, directory):
    """ Join all csvs in a directory that match a regex pattern in their filename """
    csvs = sorted(list(Path(directory).rglob(regex_pattern)))
    if len(csvs) == 0:
        raise FileNotFoundError(f"No files found in {directory} that match {regex_pattern}")
    df = pd.concat([pd.read_csv(csv) for csv in csvs])
    if "Unnamed: 0" in df.columns:
        df.drop("Unnamed: 0", axis=1, inplace=True)
    return df

In [57]:
t = join_csvs('*misc_player.csv', data_dir / 'big5')

In [64]:
test1 = pd.read_csv("/home/jimmy/Dropbox/Code/Python/FantasyFootball/data/FBRef/ENG/t1_2013_M_defense_match_stats_team.csv")

/tmp/ipykernel_19259/4205370669.py:1: DtypeWarning: Columns (22,23,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  test1 = pd.read_csv("/home/jimmy/Dropbox/Code/Python/FantasyFootball/data/FBRef/ENG/t1_2013_M_defense_match_stats_team.csv")


In [66]:
test1

,Unnamed: 0,MatchURL,League,Match_Date,Matchweek,Home_Team,Home_Formation,Home_Score,Home_xG,Home_Goals,...,SCA_SCA,GCA_SCA,Cmp_Passes,Att_Passes,Cmp_percent_Passes,PrgP_Passes,Carries_Carries,PrgC_Carries,Att_Take_Ons,Succ_Take_Ons
0,1,https://fbref.com/en/matches/e3c3ddf0/Arsenal-...,Premier League,2017-08-11,Premier League (Matchweek 1),Arsenal,3-4-3,4,2.5,Alexandre Lacazette · 2&rsquor; Danny Welbeck ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,https://fbref.com/en/matches/e3c3ddf0/Arsenal-...,Premier League,2017-08-11,Premier League (Matchweek 1),Arsenal,3-4-3,4,2.5,Alexandre Lacazette · 2&rsquor; Danny Welbeck ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,https://fbref.com/en/matches/e3c3ddf0/Arsenal-...,Premier League,2017-08-11,Premier League (Matchweek 1),Arsenal,3-4-3,4,2.5,Alexandre Lacazette · 2&rsquor; Danny Welbeck ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,https://fbref.com/en/matches/e3c3ddf0/Arsenal-...,Premier League,2017-08-11,Premier League (Matchweek 1),Arsenal,3-4-3,4,2.5,Alexandre Lacazette · 2&rsquor; Danny Welbeck ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,https://fbref.com/en/matches/e3c3ddf0/Arsenal-...,Premier League,2017-08-11,Premier League (Matchweek 1),Arsenal,3-4-3,4,2.5,Alexandre Lacazette · 2&rsquor; Danny Welbeck ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65886,65887,https://fbref.com/en/matches/b280b66a/Burnley-...,Premier League,2024-03-16,Premier League (Matchweek 29),Burnley,4-4-2,2,2.7,Jacob Bruun Larsen (P) · 10&rsquor; David Datr...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65887,65888,https://fbref.com/en/matches/8a009cc4/Fulham-T...,Premier League,2024-03-16,Premier League (Matchweek 29),Fulham,4-2-3-1,3,2.7,Rodrigo Muniz · 42&rsquor; Saša Lukić · 49&rsq...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65888,65889,https://fbref.com/en/matches/8a009cc4/Fulham-T...,Premier League,2024-03-16,Premier League (Matchweek 29),Fulham,4-2-3-1,3,2.7,Rodrigo Muniz · 42&rsquor; Saša Lukić · 49&rsq...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65889,65890,https://fbref.com/en/matches/fc356c59/West-Ham...,Premier League,2024-03-17,Premier League (Matchweek 29),West Ham United,4-2-3-1,1,2.0,Michail Antonio · 29&rsquor;,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
test1.columns

Index(['Unnamed: 0', 'MatchURL', 'League', 'Match_Date', 'Matchweek',
       'Home_Team', 'Home_Formation', 'Home_Score', 'Home_xG', 'Home_Goals',
       'Home_Yellow_Cards', 'Home_Red_Cards', 'Away_Team', 'Away_Formation',
       'Away_Score', 'Away_xG', 'Away_Goals', 'Away_Yellow_Cards',
       'Away_Red_Cards', 'Game_URL', 'Team', 'Home_Away', 'Player',
       'Player_Href', 'Player_Num', 'Nation', 'Pos', 'Age', 'Min',
       'Tkl_Tackles', 'TklW_Tackles', 'Def 3rd_Tackles', 'Mid 3rd_Tackles',
       'Att 3rd_Tackles', 'Tkl_Challenges', 'Att_Challenges',
       'Tkl_percent_Challenges', 'Lost_Challenges', 'Blocks_Blocks',
       'Sh_Blocks', 'Pass_Blocks', 'Int', 'Tkl+Int', 'Clr', 'Err',
       'Competition_Name', 'Gender', 'Country', 'Tier', 'Season_End_Year',
       'Gls', 'Ast', 'PK', 'PKatt', 'Sh', 'SoT', 'CrdY', 'CrdR', 'Touches',
       'Tkl', 'Blocks', 'xG_Expected', 'npxG_Expected', 'xAG_Expected',
       'SCA_SCA', 'GCA_SCA', 'Cmp_Passes', 'Att_Passes', 'Cmp_percent_Passes'